# SkillFactory. Data Science. Проект №3.

### Задача: предсказать рейтинг ресторанов в TripAdvisor.


# Загрузка и инициализация 

## Библиотеки

In [ ]:
import numpy as np
import pandas as pd
from collections import Counter
import re
import seaborn as sns
import missingno as missin
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import os
from datetime import datetime, timedelta
from textblob import TextBlob

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 7)
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
RANDOM_SEED = 42
!pip freeze > requirements.txt

## Датасеты

Загружаются следующие датасеты:
1. **df_train** - дата о ресторанах, на котором тренирую модель;
2. **df_test** - дата о ресторанах, на которых тестирую модель;
3. **data** - объединенный df_train и df_test, который анализирую, чищу и дополняю информацией;
4. **sample_submission** - пример отправки;
5. **cities**, **concap.csv**, **worlcities.csv** - датасеты о городах и странах, добавленные с kaggle дополнительно для создания новых признаков.


### main_task.csv, kaggle_task.csv, sample_submission.csv 
 Предоставленные датасеты, содержащие информацию о ресторанах. 

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR + '/main_task.csv')
df_test = pd.read_csv(DATA_DIR + '/kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR + '/sample_submission.csv')

Подгоняю названия полей под стандарт PEP8.

In [ ]:
for dataset in [df_train, df_test, sample_submission]:
    dataset.columns = list(map(lambda x: (x.replace(' ', '_')).lower(), dataset.columns))

Для корректной обработки признаков объединяю трейн и тест в один датасет

Помечаю трейн sample = 1

Выставляю rating = 0 в тесте нам его нужно предсказать.

In [ ]:
df_train['sample'] = 1
df_test['sample'] = 0
df_test['rating'] = 0

data = df_test.append(df_train, sort=False).reset_index(drop=True)

In [ ]:
data.info()

#### Разъяснение признаков
* **restaurant_id**: ID ресторана
* **city**: Город 
* **cuisine_style**: Кухня
* **ranking**: Ранг ресторана относительно других ресторанов в этом городе
* **price_range**: Цены в ресторане в 3 категориях
* **number_of_reviews**: Количество отзывов
* **reviews**: 2 последних отзыва и даты этих отзывов
* **url_ta**: страница ресторана на 'www.tripadvisor.com' 
* **id_ta**: ID ресторана в TripAdvisor
* **rating**: Рейтинг ресторана

In [ ]:
data.sample(5)

### worldcities.csv
Датасет для добавления таких признаков, как страна и популяция.

In [ ]:
cities = pd.read_csv('../input/world-cities/worldcities.csv')
cities = cities[['city', 'country', 'population']].copy().set_index('city')
cities.sample(10)

### countries of the world.csv
Датасет для добавления признаков о стране.

In [ ]:
countries = pd.read_csv('../input/countries-of-the-world/countries of the world.csv')
countries = countries[[
    'Country',
    'Coastline (coast/area ratio)',
    'GDP ($ per capita)',
    'Phones (per 1000)',
    'Agriculture',
    'Service'
]].copy()
countries.columns = ['country', 'coastline', 'gdp', 'phones', 'agriculture', 'service']
countries = countries.set_index('country')
countries.index = countries.index.str.strip()
countries.sample(10)

### concap.csv
Датасет для добавления признака столицы.

In [ ]:
capitals = pd.read_csv('../input/world-capitals-gps/concap.csv')
capitals = capitals[['CountryName', 'CapitalName']]
capitals.columns = ['country', 'capital']
capitals = capitals.set_index('country')
capitals.index = capitals.index.str.strip()
capitals.sample(10)

# Предобработка данных и РАД

## 0. Полезные функции

Найти самый часто встречающийся вид кухни по странам или городам.

In [ ]:
def common_cuisine(data_frame, country_or_city, by_country=True):
    column_name = 'country' if by_country else 'city'
    return data_frame[data_frame[column_name] == country_or_city]['cuisine_style'].value_counts().index[0]

Распарсить ревью поля

In [ ]:
def parse_reviews(df):
    df['reviews'].fillna('[[], []]', inplace=True)
    df['tmp_reviews'] = df['reviews'].apply(lambda x: x.split('],')[0].split("',"))

    df['review_1'] = df['tmp_reviews'].apply(lambda x: x[0])
    df['review_2'] = df['tmp_reviews'].apply(lambda x: x[-1])
    df['review_1'] = df['review_1'].apply(
        lambda x: x.replace('[', '').replace("'", '').replace('"', '')
    )
    df['review_2'] = df['review_2'].apply(
        lambda x: x.replace('[', '').replace("'", '').replace('"', '')
    )
    df.drop('tmp_reviews', axis=1, inplace=True)

    # remove duplicates
    df['review_2'] = df[['review_2', 'review_1']].apply(
        lambda x: '' if x[0] == x[1] else x[0],
        axis=1
    )
    
    df['review_ton_1'] = df['review_1'].apply(lambda x: TextBlob(x).polarity)
    df['review_ton_2'] = df['review_2'].apply(lambda x: TextBlob(x).polarity)

    df['tmp_days_reviews'] = df['reviews'].apply(lambda x: x.split('],')[-1].split("',"))
    df['day_1'] = df['tmp_days_reviews'].apply(lambda x: x[0])
    df['day_2'] = df['tmp_days_reviews'].apply(lambda x: x[-1])
    df['day_1'] = df['day_1'].apply(
        lambda x: x.replace('[', '').replace("'", '').replace(']', '').replace(' ', '').replace('"', '')
    )
    df['day_2'] = df['day_2'].apply(
        lambda x: x.replace('[', '').replace("'", '').replace(']', '').replace(' ', '').replace('"', '')
    )
    df.drop('tmp_days_reviews', axis=1, inplace=True)

    # remove duplicates
    df['day_2'] = df[['day_2', 'day_1']].apply(
        lambda x: '' if x[1] == '' else x[0],
        axis=1
    )

    # format dates
    df['day_1'] = df['day_1'].apply(lambda x: x if len(x) == 0 else datetime.strptime(x, '%m/%d/%Y'))
    df['day_2'] = df['day_2'].apply(lambda x: x if len(x) == 0 else datetime.strptime(x, '%m/%d/%Y'))
    
    df['reviews_days_diff'] = abs(df['day_1'] - df['day_2']).apply(lambda x: x.days)

Вычислить разницу в количестве днях между двумя входящими датами.

Подсчитать квартили и выбросы IQR для поля

In [ ]:
def iqr_for_column(data, column):
    perc_25 = data[column].quantile(0.25)
    perc_75 = data[column].quantile(0.75)
    iqr = perc_75 - perc_25
    min_out = perc_25 - 1.5 * iqr
    max_out = perc_75 + 1.5 * iqr
    anomaly = len(data[data[column] > max_out]) + len(data[data[column] < min_out])
    print(
        '25-й перцентиль: {} |'.format(perc_25),
        '75-й перцентиль: {} |'.format(perc_75),
        "IQR: {} | ".format(iqr),
        "Границы выбросов: [{}, {}].".format(min_out, max_out))
    print(
        "Выбросов, согласно IQR: {} | {:2.2%}".format(
            anomaly,
            anomaly / len(data)
        )
    )

## 1. Обработка неизвестных значений NaN и создание новых признаков.
### 1.1 Перед обработкой NAN  выношу информацию о наличии пропусков как отдельный признак.  

Отображаю матрицу неизвестных значений.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
missin.matrix(data, ax=ax, sparkline=False)

Нахожу столбцы с неизвестными значениями.

In [ ]:
cols_with_nans = (data.isnull().any()[lambda column: column]).index
cols_with_nans

Создаю новый бинарный признак для каждого столбца с неизвестными значениями о наличии пропусков.

In [ ]:
for col in cols_with_nans:
    new_col_name = col + '_is_nan'
    data[new_col_name] = pd.isna(data[col]).astype('uint8')

### 1.2 Разбор столбцов с NaN по отдельности: замена NaN значений и создание взаимосвязанных новых признаков.
Неизвестные значения имеются в стобцах **cuisine_style**, **price_range**, **number_of_reviews** и **reviews**.

Рассмотриваю каждый из них по отдельности.

Также, дополнительно создаю новые признаки такие, как **country** и **cuisine_count**.

#### Признак country

Создаю и заполняю новый признак country в датасете по городам.

In [ ]:
data['country'] = pd.Series(np.array([None] * data.shape[0]))

for name in data.city.unique():
    if name in cities.index:
        try:
            data.loc[(data.city == name), 'country'] = cities.loc[name].country
        except:
            data.loc[(data.city == name), 'country'] = cities.loc[name].country.values[0]

Проверяю заполнен ли country полностью.

In [ ]:
data[data.country.isnull()]

Заполняю вручную отстутствующие значения для новых признаков.

In [ ]:
data.loc[(data.city == 'Oporto'), 'country'] = 'Portugal'
data.loc[(data.city == 'Zurich'), 'country'] = 'Switzerland'
data.loc[(data.city == 'Krakow'), 'country'] = 'Poland'

data[data.country.isnull()]

#### Признак cuisine_style и cuisine_count

**23% (11590) неизвестных значений** 

В данном столбце каждая ячейка - строка.

Преобразую формат ячеек из строки в настоящий лист со строками.

Посчитаю процентное соотношение неизвестных значений.

In [ ]:
stats = data['cuisine_style'].isnull().value_counts()
print(stats)
print(100 * stats[1] / (stats[0] + stats[1]))

Привожу формат ячейки из формата строки в формат листа, состоящего из строк.

In [ ]:
df_cuisine_style = pd.DataFrame(data['cuisine_style'])
df_cuisine_style['cuisine_style'] = df_cuisine_style['cuisine_style'].str.strip('[]')
df_cuisine_style['cuisine_style'] = df_cuisine_style['cuisine_style'].str.replace('\'', '')
data['cuisine_style'] = df_cuisine_style['cuisine_style'].str.split(', ')

Чтобы найти самую часто встречающуюся кухню нужно использовать ***explode***.
Для этого создаю копию датасета df_cuisine_style_explode, чтобы не повлияло на оригинал.

In [ ]:
df_cuisine_style_explode = data.explode('cuisine_style')
df_cuisine_style_explode.sample(5)

In [ ]:
data[data.cuisine_style_is_nan == 1].head()

Заполняю неизвестные значения самой популярной кухней в городе. Можно попробовать по странам, но лучше распределение получилось по городам.

In [ ]:
for name in data.city.unique():
    cuisine = common_cuisine(df_cuisine_style_explode, name, by_country=False)
    data.loc[(data.city == name) & (data.cuisine_style_is_nan == 1), 'cuisine_style'] = cuisine

In [ ]:
data.cuisine_style_is_nan.value_counts()

In [ ]:
data.cuisine_style.isnull().value_counts()

In [ ]:
data[data.cuisine_style_is_nan == 1].head()

#### Новый признак cuisine_count
Создаю новый признак - количество видов кухонь предлагаемых в ресторане.
Если значение **cuisine_style** неизвестно, то по дефолту **cuisine_count** равен 1.

In [ ]:
data['cuisine_count'] = data['cuisine_style'].apply(
    lambda cuisines: 1 if str(cuisines) == 'nan' else len(cuisines)
)
data[['cuisine_style', 'cuisine_count']].head(10)

#### Признак price_range 

**34.7% (17361) неизвестных значений.** 
 
Так как 70.6% ресторанов в ценовом диапазоне от  10-999 долларов, то все неизвестные значения ресторанов будут заменены на этот ценовой диапазон. 

In [ ]:
stats = data.price_range.isna().value_counts()
stats[1] * 100 / (stats[0] + stats[1])

In [ ]:
common_price = data.price_range.value_counts().index[0]

In [ ]:
y = data.price_range.value_counts().values
graph = plt.bar(['Средняя цена', 'Дешево', 'Дорого'], y, color='pink')

In [ ]:
data.price_range.fillna(common_price, inplace=True)
data.price_range 

#### Признак reviews
**2 незивестных значения.**

Без ревью нет рейтинга, а целевая переменная - рейтинг.

Соответсвенно нам такие данные не нужны. Можно их удалить.

In [ ]:
display(data[data['reviews_is_nan'] == 1])

Также удаляем поле reviews_is_nan т.к. оно больше ни на что не влияет

In [ ]:
data['reviews'] = data['reviews'].dropna(axis=0)
del data['reviews_is_nan']

#### Новые признаки review_ton_1, review_ton_2, reviews_days_diff
Создаю новые признаки:

review_ton_1, review_ton_2 - тональные окраски двух последних ревью

review_days_diff - количество дней прошедших между ними

Для этого необходим распарсить строковые значения и перевести в формат даты. 

In [ ]:
parse_reviews(data)
data.sample(5)

In [ ]:
data.sort_values(by=['reviews_days_diff']).head(5)

#### Признак number_of_reviews
6.4% (3200) неизвестных значений. 

Между **number_of_reviews** и **cuisine_count** имеется средняя позитивная корреляция 0.4. Также, между **number_of_reviews** и **ranking** существует слабая негативная корреляция -0.2. График медианного значения кол-ва отзывов по кол-ву видов кухни показывает то, что в принципе медианное количество отзывов разнится между ресторанами, предлагающими разное количество видов кухонь. В связи с чем заполняю неизвестные значения ресторана медианным значением той группы ресторанов, сгруппированных по кол-ву видов кухни, в которую он квалифицируется. 

In [ ]:
data[data['number_of_reviews_is_nan'] == 1].shape[0] / data.shape[0] * 100

In [ ]:
sns.set(rc={'figure.figsize': (11, 8)}, font_scale=1.5, style='whitegrid')
print(data.select_dtypes('number').corr().loc['number_of_reviews'].sort_values())

sns.heatmap(data.select_dtypes('number').corr())

In [ ]:
plt.scatter(data.number_of_reviews, data.ranking)
plt.title("ranking vs number_of_reviews")
plt.xlabel("number_of_reviews")
plt.ylabel("ranking")
plt.show()

In [ ]:
plt.scatter(data.cuisine_count, data.number_of_reviews)
plt.title("cuisine_count vs number_of_reviews")
plt.xlabel("cuisine_count")
plt.ylabel("number_of_reviews")
plt.show()

In [ ]:
data.number_of_reviews.describe()

Разница между средним значением и медианой большая: есть большие выбросы, влияющие сильно на среднее значение. 

In [ ]:
print(data.number_of_reviews.mean())
print(data.number_of_reviews.median())
print(data.number_of_reviews.std())

In [ ]:
# Вычислить медиану для каждой группы рестроранов, сгруппированных по кол-ву предлагаемых видов кухни. 
reviews_count_data = pd.DataFrame(data.groupby('cuisine_count').median().number_of_reviews)
plot = reviews_count_data.plot.bar(title='Медианное значение кол-ва отзывов по кол-ву видов кухни')
for p in plot.patches:
    plot.annotate(str(int(p.get_height())), (p.get_x(), p.get_height()), color='purple')

In [ ]:
# Заполнить неизвестные значения медианой соответсвенно группе, которой принадлежит ресторан. 
data.number_of_reviews = data.apply(
    lambda row: (reviews_count_data.loc[row.cuisine_count])[0]
    if np.isnan(row.number_of_reviews) else row.number_of_reviews,
    axis=1
)

### 1.3 Категоризация признаков

#### 1.3.1 Категориальные (номинативные) признаки.

In [ ]:
# Выбрать только нечисловые столбцы. 
nominative = data.select_dtypes('object')
nominative.sample(3)

#### 1.3.2 Ординальные признаки

Ординальные признаками являются **ranking**, **price_range**, **rating**, так как их можно по смыслу расставить по возрастанию. 

In [ ]:
data.price_range = data.price_range.map({'$': 0, '$$ - $$$': 1, '$$$$': 2})

#### 1.3.3 Числовые признаки

In [ ]:
# Выбрать только числовые столбцы. 
numerical = data.select_dtypes('number')
numerical.sample(3)

## Feature Engineering, обработка новых признаков и РАД

Создаю новые признаки, путем добавления информации из других датасетов, импортированных ранее. 

### Новые признаки **capital**, **is_capital**, **population**, **coastline**, **gdp**, **phones**, **agriculture**, **service**

In [ ]:
new_cols = ['country', 'capital', 'population', 'coastline', 'gdp', 'phones', 'agriculture', 'service']
for col in new_cols:
    data[col] = pd.Series(np.array([None] * data.shape[0]))

# Заполнить значения новых признаков в главном датасете с помощью датасета о городах. 
for city_name in data.city.unique():
    if city_name in cities.index:
        try:
            data.loc[(data.city == city_name), 'country'] = cities.loc[city_name].country
            data.loc[(data.city == city_name), 'population'] = int(cities.loc[city_name].population)
        except:
            data.loc[(data.city == city_name), 'country'] = cities.loc[city_name].country.values[0]
            data.loc[(data.city == city_name), 'population'] = int(cities.loc[city_name].population.values[0])

for country_name in data.country.unique():
    if country_name in capitals.index:
        data.loc[data.country == country_name, 'capital'] = capitals.loc[country_name].capital
    if country_name in countries.index:
        for col in ['coastline', 'gdp', 'phones', 'agriculture', 'service']:
            data.loc[data.country == country_name, col] = countries.loc[country_name][col]
data[new_cols]

In [ ]:
to_change = ['coastline', 'phones', 'agriculture', 'service', 'gdp']
for col in to_change:
    data[col] = pd.to_numeric(data[col].astype(str).str.replace(',', '.'), errors='coerce')

In [ ]:
data[new_cols]

In [ ]:
# Проверить в каких новых признаках имеются неизвестные значения.
fig, ax = plt.subplots(figsize=(8, 8))
missin.matrix(data, ax=ax, sparkline=False)

In [ ]:
# Заполнить вручную отстутствующие значения для популяции и стран.
data.loc[(data.city == 'Oporto'), 'population'] = 214349
data.loc[(data.city == 'Zurich'), 'population'] = 402762
data.loc[(data.city == 'Krakow'), 'population'] = 766683

data.loc[(data.city == 'Oporto'), 'country'] = 'Portugal'
data.loc[(data.city == 'Zurich'), 'country'] = 'Switzerland'
data.loc[(data.city == 'Krakow'), 'country'] = 'Poland'

In [ ]:
# Найти для каких стран нет информации по столицам,ввп и соотношению береговой линии и заполнить вручную.
missing = data[data.capital.isnull()]['country'].unique()
missing_info = {
    'Czechia': ['Prague', 23495, 0],
    'Portugal': ['Lisbon', 23252, 0.98],
    'Poland': ['Warsaw', 15693, 0],
    'Switzerland': ['Bern', 81994, 0]
}

for country in missing:
    for i, col in zip([0, 1, 2], ['capital', 'gdp', 'coastline']):
        data.loc[(data.country == country), col] = missing_info[country][i]

In [ ]:
# Создать признак is_capital
new_cols.append('is_capital')
data['is_capital'] = pd.Series(np.array([0] * data.shape[0]))
for capital_name in data.capital.unique():
    data.loc[(data.city == capital_name), 'is_capital'] = 1

In [ ]:
data[['country', 'city', 'capital', 'is_capital']].sample(10)

Сервис и агрокультура не сильно разнятся по странам или ВВП, поэтому заполняю пропущенные значения средним значением. 

In [ ]:
figure, axes = plt.subplots(2, 1, figsize=(15, 15))
figure.tight_layout(pad=9.0)

data.groupby('country')['service', 'agriculture'].mean().plot.bar(
    ax=axes[0],
    rot=90,
    title='Развитость по сервису vs агрокультуре по странам'
)
data.groupby('gdp')['service', 'agriculture'].mean().plot.bar(
    ax=axes[1],
    rot=90,
    title='Развитость по сервису vs агрокультуре по ВВП'
)

In [ ]:
data['service'].fillna(data['service'].mean(), inplace=True)
data['agriculture'].fillna(data['agriculture'].mean(), inplace=True)

Пропущенные значения кол-ва телефонов заменяю медианой, так как значения по странам и ВВП разнятся.  

In [ ]:
figure, axes = plt.subplots(2, 1, figsize=(15, 15))
figure.tight_layout(pad=10)
data.groupby('country')['phones'].mean().plot.bar(
    ax=axes[0],
    rot=90,
    title='Кол-во телефонов на 1000 человек по странам',
    color='orange'
)
data.groupby('gdp')['phones'].mean().plot.bar(
    ax=axes[1],
    rot=90,
    title='Кол-во телефонов на 1000 человек по ВВП',
    color='purple'
)
plt.show()

In [ ]:
data['phones'].fillna(data['phones'].median(), inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
missin.matrix(data, ax=ax, sparkline=False)

Осталось два неизвестных значения, но с ними ничего не делаю, так как они в признаке reviews, которые в модель МО не будет отправляться.

In [ ]:
data[data.isnull().any(axis=1)]

### Более подробный анализ признаков **ranking**, так как он должен быть связан с целевой переменной

In [ ]:
# Посторить распределение ранга ресторанов в общем.
data['ranking'].hist(bins=100)

In [ ]:
# Рассмотреть количество ресторанов по городам и странам
figure, axes = plt.subplots(1, 2, figsize=(15, 15))
figure.tight_layout(pad=4)
data['city'].value_counts(ascending=True).plot(kind='barh', ax=axes[0])
data['country'].value_counts(ascending=True).plot(kind='barh', ax=axes[1])

In [ ]:
# Рассмотреть распределение ресторанов одного города. 
data['ranking'][data['city'] == 'London'].hist(bins=100)

Из гистограммы ниже следует то,что вероятней всего ранг ресторанов не глобальный, а локальный, то есть в каждом городе в отдельности. 

In [ ]:
def show_distr(n, by='city', of='ranking', title=''):
    top_n = (data[by].value_counts())[0:n].index
    for x in top_n:
        plot = data[of][data[by] == x].plot(kind='hist', legend=True, bins=100, title=title)
    plot.legend(top_n)
    plt.show()


def normalise(data, by, feature):
    feature_maxes = data.groupby(by)[feature].max()
    return data[[by, feature]].apply(lambda row: row[feature] / feature_maxes.loc[row[by]], axis=1)


show_distr(10, title='Распределение ранга ресторанов городов топ-10 по кол-ву ресторанов')

In [ ]:
# Нормализовать переменную с целью подведения под одну шкалу ранг и создать новый признак
data['normalised_rank'] = data['ranking']
data['normalised_rank'] = normalise(data, 'city', 'ranking')
data['normalised_rank']

In [ ]:
# Показать распределение нового признака нормализованный ранг. 
show_distr(10, 'city', 'normalised_rank', title='Нормализованный ранг ресторанов топа-10 городов')

### Анализ целевой переменной **rating**

In [ ]:
# Постоить гистограмму распределения рейтинга ресторанов.
stats = data['rating'].value_counts(ascending=True).sort_index()
plot = stats.plot(kind='bar', color='orange', title='Распределение рейтинга ресторанов')
for p in plot.patches:
    perc = round(int(p.get_height()) / sum(stats.values) * 100, 1)
    plot.annotate(str(perc) + '%', (p.get_x(), p.get_height()), color='purple')

In [ ]:
# Построить гистограмму, показувающее соотношение между столичными и ресторанами других городов, а также рейтингом. 
figure, axes = plt.subplots(2, 1, figsize=(15, 15))
figure.tight_layout(pad=4)

# Определить условия в отдельные переменные для удобства.
rating_is_5 = data['rating'] == 5
good_rating = data['rating'] >= 4
is_capital = (data['is_capital'] == 1)
conds = [
    [is_capital & ~rating_is_5, ~is_capital & ~rating_is_5, is_capital & rating_is_5, ~is_capital & rating_is_5],
    [is_capital & good_rating, is_capital & ~good_rating, ~is_capital & good_rating, ~is_capital & ~good_rating]
]

# Вручную составить по соответсвенному порядку названия графиков на одной гистограмме для удобства понимания.
legends = [['столица, рейтинг<5', 'не столица, рейтинг<5', 'столица, рейтинг=5', 'не столица, рейтинг=5'],
           ['столица, рейтинг>=4', 'столица, рейтинг<4', 'не столица, рейтинг>=4', 'не столица, рейтинг<4']]

# Нарисовать графики
for i in range(2):
    for cond, clr in zip(conds[i], ['pink', 'orange', 'green', 'purple']):
        other_tail = data[cond]['ranking'].plot(kind='hist', legend=True, ax=axes[i], bins=100, color=clr)
    other_tail.legend(labels=legends[i])


# Подготовка

In [ ]:
md = data.select_dtypes('number').drop_duplicates()

In [ ]:
md

In [ ]:
from sklearn.preprocessing import StandardScaler

scaled_md = md
scaled_md.info()

scaler = StandardScaler()
scaled = scaler.fit_transform(scaled_md)
scaled_md = pd.DataFrame(scaled)
scaled_md.columns = md.columns
scaled_md['sample'] = md['sample']
scaled_md['rating'] = md['rating']

In [ ]:
scaled_md['sample']

In [ ]:
sns.heatmap(scaled_md.drop(['sample'], axis=1).corr(), )


# Обучение модели

In [ ]:
data.shape

In [ ]:
scaled_md.columns

In [ ]:
def prepare_model_data(md):
    features = [
        'ranking',
        'price_range',
        'number_of_reviews',
        'cuisine_style_is_nan',
        'price_range_is_nan',
        'number_of_reviews_is_nan',
        'cuisine_count',
        'review_ton_1',
        'review_ton_2',
        'reviews_days_diff',
        'population',
        'coastline',
        'gdp',
        'phones',
        'agriculture',
        'service',
        'is_capital',
        'normalised_rank',
        'sample',
        'rating',
    ]

    result = md[features]
    to_encode = data['city']
    result = pd.concat([result, pd.get_dummies(to_encode)], axis=1)
    result = result.apply(pd.to_numeric, errors='coerce')
    for i in result.columns:
        result.fillna(result[i].mean(), inplace=True)
    return result

#### Запускаем и проверяем что получилось

In [ ]:
# Теперь выделим тестовую часть
scaled_md = prepare_model_data(scaled_md)
train_data = scaled_md.query('sample == 1').drop(['sample'], axis=1)
test_data = scaled_md.query('sample == 0').drop(['sample'], axis=1)

y = train_data.rating.values  # наш таргет
X = train_data.drop(['rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor  # инструмент для создания и обучения модели
from sklearn import metrics  # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10, 10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission


In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['rating'], axis=1)

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['rating'] = predict_submission
sample_submission.columns = ['Restaurant_id', 'Rating']
sample_submission.to_csv('./submission.csv', index=False)
sample_submission.head(10)